# Projet DALAS
## Données entre 2014-2024 (exclu)
### SOYKOK Aylin 28711545 - CELIK Simay 28713301


<b>
Cette notebook contient le code afin de scraper des données de "quality of life" pour plusieurs pays entre plusieurs années depuis le site numbeo. Le document est en franglais, il faut le corriger.

TO DO:
data cleaning

In [1]:
import bs4
import lxml
import pandas as pd
import urllib

from urllib import request

C:\Users\PC\AppData\Local\Temp\ipykernel_21092\633652058.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
years = [i for i in range(2014,2024)]
link = "https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title=2024"
req = request.urlopen(link).read()
page = bs4.BeautifulSoup(req, "lxml")
table = page.find('table',{"id":"t2"})
headers = [th.text.rstrip() for th in table.find_all('th')]
headers.append("Year")
rows = table.find_all('tr')
list_data = []
for row in rows[1:]:
  data = [d.text.rstrip() for d in row.find_all('td')]
  data.append(2024)
  list_data.append(data)

for year in years:
  link = "https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title="+str(year)
  req = request.urlopen(link).read()
  page = bs4.BeautifulSoup(req, "lxml")
  table = page.find('table',{"id":"t2"})
  rows = table.find_all('tr')
  for row in rows[1:]:
    data = [d.text.rstrip() for d in row.find_all('td')]
    data.append(year)
    list_data.append(data)

In [18]:
df = pd.DataFrame(list_data,columns=headers)
df.columns

Index(['Rank', 'Country', 'Quality of Life Index', 'Purchasing Power Index',
       'Safety Index', 'Health Care Index', 'Cost of Living Index',
       'Property Price to Income Ratio', 'Traffic Commute Time Index',
       'Pollution Index', 'Climate Index', 'Year'],
      dtype='object')

In [9]:
df_nan =  df.isna()
nan_lines = df_nan.any(axis=1)
nan_lines[nan_lines==True]

Series([], dtype: bool)

Aucun NaN.

For missing values (some countries do not exist in some years) we can detect which country is missing in which year and then for all the columns we can put the mean of the previous and the next year. Using adjacent years would prevent weird points in a graph. We can also mark these "estimations" to know which were generated by us and which were in the original source. => Ask the prof.

<b> Cleaning missing values


In [10]:
df['Generated'] = 0 #1 si généré par nous, 0 sinon

def fill_missing(group):
    """Trouve les lignes manquantes, les remplit avec la moyenne des années adjacentes."""
    missing = set(df['Country'].unique()) - set(group['Country'].unique()) #pays manquants par groupe d'année
    for m in missing:
        prev = df[(df['Country']==m) & (df['Year']==group['Year'].min() - 1)]
        next = df[(df['Country']==m) & (df['Year']==group['Year'].max() + 1)]
        if not prev.empty and not prev.empty:
            moy = (prev.iloc[:, 1:-1] + prev.iloc[:, 1:-1]) / 2
            moy['Year'] = group['Year']
            moy['Country'] = m
            moy['Generated'] = 1 #1 si généré par nous
            group = pd.concat([group, moy], ignore_index=True)
    return group
        

filled_df = df.groupby('Year').apply(fill_missing).reset_index(drop=True)
filled_df

In [23]:
df.to_csv('../data/data_annees.csv', index=False)